<a href="https://colab.research.google.com/github/ykitaguchi77/Strabismus_AI_project/blob/main/DataSplit(stratified_one_subject_leave_out).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Data_split for one-subject-leave-out stratified 5-fold crossvalidation**

In [ ]:
"""
Leave one subject out cross validation + 5-fold stratified cross validation

・1症例を抜き出し、その症例のすべての画像をテスト画像とする
・残りの症例の内斜視、外斜視、斜視なし群を、同じ症例が群をまたがないように5分割する。
・5分割したデータセットのうち4つをtraining、1つをvalidationとして用いてトレーニングを行い、抜き出した1症例のそれぞれの画像のおける正解率を算出する。これを5回繰り返してcross validationとする。

"""

In [16]:
import codecs
import pandas as pd
import os
pd.set_option('display.max_rows', 500)


gla_ortho_path = r"F:\先天性緑内障\data_list\Disease_ortho.csv"
gla_eso_path = r"F:\先天性緑内障\data_list\Disease_ET.csv"
gla_exo_path = r"F:\先天性緑内障\data_list\Disease_XT.csv"
cont_ortho_path = r"F:\先天性緑内障\data_list\Control_ortho.csv"
cont_eso_path = r"F:\先天性緑内障\data_list\Control_ET.csv"
cont_exo_path = r"F:\先天性緑内障\data_list\Control_XT.csv"

def opencsv(path, classname):
    #with codecs.open(path, "r", "utf-8", "ignore") as file:
    with codecs.open(path, "r", "Shift-JIS", "ignore") as file:
        df = pd.read_csv(file, index_col=None, header=None)
        df.insert(0, 'ID', '')
        df.insert(0, 'classes', '')
        for row in range(len(df)):
            df.iloc[row,0] = classname #class
            df.iloc[row,1] = os.path.basename(df.iloc[row,2]).split("_")[0] #ID
    return df

df_gla_ortho = opencsv(gla_ortho_path, "gla_ortho") 
df_gla_eso = opencsv(gla_eso_path, "gla_eso")
df_gla_exo = opencsv(gla_exo_path, "gla_exo")
df_cont_ortho = opencsv(cont_ortho_path, "cont_ortho")
df_cont_eso = opencsv(cont_eso_path, "gla_eso")
df_cont_exo = opencsv(cont_exo_path, "gla_exo")

df_gla_all = pd.concat([df_gla_ortho, df_gla_eso, df_gla_exo], axis=0)
df_cont_all = pd.concat([df_cont_ortho, df_cont_eso, df_cont_exo], axis=0)
df_all = pd.concat([df_cont_ortho, df_cont_eso, df_cont_exo], axis=0)

#df_all.to_csv(r"F:\先天性緑内障\data_list\df_all.csv", encoding='utf-8-sig', index=0, header=None)

In [34]:
#それぞれの項目（path, classes, ID）をリスト化
gla_dataset_path = df_gla_all.iloc[:,2].transpose().values
gla_classes = df_gla_all.iloc[:,0].transpose().values
gla_id = df_gla_all.iloc[:,1].transpose().values
cont_dataset_path = df_cont_all.iloc[:,2].transpose().values
cont_classes = df_cont_all.iloc[:,0].transpose().values
cont_id = df_cont_all.iloc[:,1].transpose().values

print(len(gla_dataset_path))
print(len(gla_classes))
print(len(gla_id))

logo = LeaveOneGroupOut()
logo.get_n_splits(gla_dataset_path, gla_classes, gla_id)
logo.get_n_splits(groups=gla_id)  # 'groups' is always required

for train_index, test_index in logo.split(gla_dataset_path, gla_classes, gla_id):
     #print("TRAIN:", train_index, "TEST:", test_index)
     gla_dataset_path_train, gla_dataset_path_test = gla_dataset_path[train_index], gla_dataset_path[test_index]
     gla_id_train, gla_id_test = gla_id[train_index], gla_id[test_index]
     #print(gla_dataset_path, gla_dataset_path_test, gla_id_train, gla_id_test)
print(gla_id_test)

"""
cv = StratifiedGroupKFold(n_splits=5)
for train_idxs, test_idxs in cv.split(gla_dataset_path, gla_classes, gla_id):
    print("TRAIN:", gla_classes[train_idxs])
    print("      ", gla_id[train_idxs])
    print("      ", gla_dataset_path[train_idxs])
    print(" TEST:", gla_classes[test_idxs])
    print("      ", gla_id[test_idxs])
    print("      ", gla_dataset_path[train_idxs])
"""

204
204
204
['9201' '9201' '9201' '9201' '9201' '9201' '9201' '9201' '9201']


'\ncv = StratifiedGroupKFold(n_splits=5)\nfor train_idxs, test_idxs in cv.split(gla_dataset_path, gla_classes, gla_id):\n    print("TRAIN:", gla_classes[train_idxs])\n    print("      ", gla_id[train_idxs])\n    print("      ", gla_dataset_path[train_idxs])\n    print(" TEST:", gla_classes[test_idxs])\n    print("      ", gla_id[test_idxs])\n    print("      ", gla_dataset_path[train_idxs])\n'

In [ ]:
# one group leave out 
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html#sklearn.model_selection.LeaveOneGroupOut
# 今回のケースでは、groupがIDに該当
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]])
y = np.array([1, 2, 1, 2, 1, 1])
groups = np.array([1, 1, 2, 3, 3, 4])
logo = LeaveOneGroupOut()
logo.get_n_splits(X, y, groups)
logo.get_n_splits(groups=groups)  # 'groups' is always required
print(logo)
LeaveOneGroupOut()
for train_index, test_index in logo.split(X, y, groups):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]
     print(X_train, X_test, y_train, y_test)

LeaveOneGroupOut()
TRAIN: [2 3 4 5] TEST: [0 1]
[[ 5  6]
 [ 7  8]
 [ 9 10]
 [11 12]] [[1 2]
 [3 4]] [1 2 1 1] [1 2]
TRAIN: [0 1 3 4 5] TEST: [2]
[[ 1  2]
 [ 3  4]
 [ 7  8]
 [ 9 10]
 [11 12]] [[5 6]] [1 2 2 1 1] [1]
TRAIN: [0 1 2 5] TEST: [3 4]
[[ 1  2]
 [ 3  4]
 [ 5  6]
 [11 12]] [[ 7  8]
 [ 9 10]] [1 2 1 1] [2 1]
TRAIN: [0 1 2 3 4] TEST: [5]
[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]] [[11 12]] [1 2 1 2 1] [1]


In [ ]:
# one group leave out 
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html#sklearn.model_selection.LeaveOneGroupOut
# 今回のケースでは、groupがIDに該当
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]])
y = np.array([1, 2, 1, 2, 1, 1])
groups = np.array([1, 1, 2, 3, 3, 4])
logo = LeaveOneGroupOut()
logo.get_n_splits(X, y, groups)
logo.get_n_splits(groups=groups)  # 'groups' is always required
print(logo)
LeaveOneGroupOut()
for train_index, test_index in logo.split(X, y, groups):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]
     print(X_train, X_test, y_train, y_test)

LeaveOneGroupOut()
TRAIN: [2 3 4 5] TEST: [0 1]
[[ 5  6]
 [ 7  8]
 [ 9 10]
 [11 12]] [[1 2]
 [3 4]] [1 2 1 1] [1 2]
TRAIN: [0 1 3 4 5] TEST: [2]
[[ 1  2]
 [ 3  4]
 [ 7  8]
 [ 9 10]
 [11 12]] [[5 6]] [1 2 2 1 1] [1]
TRAIN: [0 1 2 5] TEST: [3 4]
[[ 1  2]
 [ 3  4]
 [ 5  6]
 [11 12]] [[ 7  8]
 [ 9 10]] [1 2 1 1] [2 1]
TRAIN: [0 1 2 3 4] TEST: [5]
[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]] [[11 12]] [1 2 1 2 1] [1]


In [ ]:
# example of stratified group Kfold
# 今回のケースでは、groupがID、yがclassesに該当
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
X = np.ones((17, 2))
y = np.array([0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
groups = np.array([1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 5, 5, 6, 6, 7, 8, 8])
cv = StratifiedGroupKFold(n_splits=3)

print(X)
print(y)
print(groups)

for train_idxs, test_idxs in cv.split(X, y, groups):
    print("TRAIN:", groups[train_idxs])
    print("      ", y[train_idxs])
    print(" TEST:", groups[test_idxs])
    print("      ", y[test_idxs])

In [25]:
# one group leave out 
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html#sklearn.model_selection.LeaveOneGroupOut
# 今回のケースでは、groupがIDに該当
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]])
y = np.array([1, 2, 1, 2, 1, 1])
groups = np.array([1, 1, 2, 3, 3, 4])
logo = LeaveOneGroupOut()
logo.get_n_splits(X, y, groups)
logo.get_n_splits(groups=groups)  # 'groups' is always required
print(logo)
LeaveOneGroupOut()
for train_index, test_index in logo.split(X, y, groups):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]
     print(X_train, X_test, y_train, y_test)

LeaveOneGroupOut()
TRAIN: [2 3 4 5] TEST: [0 1]
[[ 5  6]
 [ 7  8]
 [ 9 10]
 [11 12]] [[1 2]
 [3 4]] [1 2 1 1] [1 2]
TRAIN: [0 1 3 4 5] TEST: [2]
[[ 1  2]
 [ 3  4]
 [ 7  8]
 [ 9 10]
 [11 12]] [[5 6]] [1 2 2 1 1] [1]
TRAIN: [0 1 2 5] TEST: [3 4]
[[ 1  2]
 [ 3  4]
 [ 5  6]
 [11 12]] [[ 7  8]
 [ 9 10]] [1 2 1 1] [2 1]
TRAIN: [0 1 2 3 4] TEST: [5]
[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]] [[11 12]] [1 2 1 2 1] [1]
